In [3]:
#https://www.youtube.com/watch?v=K_WbsFrPUCk 
#Speech recognition source
#text to speech from Stringer example
#https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python#question2
#to be able to talk into microphone
import speech_recognition as sr
#to be able to convert text and have speaker say it 
from os import system as sys
#to extract data from sites
import requests 
#to extract import html components from sites
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import requests
from IPython.display import display, HTML
#for google/amazon searches
import webbrowser
import json


#make dictionary of locations and match their time stamps w values
def GetDestinyUsa():
    response = requests.get('https://www.syracuse.com/destiny-usa/index.ssf/2014/07/destiny_usa_hours_syracuse_mall_stores_restaurants_entertainment_directory.html')
    data = pd.read_html(website)
    characters = data_dict['results']
    return characters


#computer speak and sometimes print what is said
def say_this(text,print_):
    sys("say %s" % (text))
    if print_ != "n":
        print(text)
        
#Getting Stock information 
#Getting html from nasdaq for stocks
def get_Nasdaq_html(symbol):
    url = 'http://www.nasdaq.com/symbol/' + symbol
    response = requests.get(url)
    if response.ok:
        return response.text
    else:
        return "Error retrieving " + url
    
#Taking useful information out of stock html 
def extract_stock_data(html):
    soup = BeautifulSoup(html, "lxml")
    name = soup.select("div#qwidget_pageheader h1")[0].text
    price = soup.select("div#qwidget_lastsale")[0].text
    change = soup.select("div#qwidget_percent")[0].text
    stock= { 'Name' : name,
            "Price" : price,
            "Change" : change }
    return stock

def find_stock_info(symbol):
    html = get_Nasdaq_html(symbol)
    stock = extract_stock_data(html)
    print("Name: %s" % stock['Name'],'y')
    print("Price: %s" % stock['Price'])
    print("Change: %s" % stock['Change'])

#https://openweathermap.org/api
def get_local_weather():
    weather_address = 'http://api.openweathermap.org/data/2.5/weather?appid=a078822a1b9ec351c0bf4c4534f3836d&q='
    url = weather_address + "Syracuse"
    data = requests.get(url).json()
    description = data['weather'][0]['description']
    temp_kelvin = data['main']['temp']
    temp_farhen = 9/5 * (temp_kelvin - 273) + 32
    temp_celc = temp_kelvin - 273
    say_this("The weather in Syracuse, NY is %2.f degrees fahrenheit and %2.f degrees celcius. The forcast is looking like %s." % (temp_farhen, temp_celc, description),'y')
    

website = 'https://www.syracuse.com/destiny-usa/index.ssf/2014/07/destiny_usa_hours_syracuse_mall_stores_restaurants_entertainment_directory.html#shop-a'
data = pd.read_html(website)
#print(type(data[0]))
data[0].set_index("Name", inplace = True)
data[1].set_index("Name", inplace = True)
data[2].set_index("Name", inplace = True)
pd.set_option('display.max_rows', 132)
#display(data[0])
          
x = data[0].count()
#print(x)

    
r = sr.Recognizer()
tasking_is_on = True


with sr.Microphone() as source:
    say_this("Hi my Name is Digimall. I can give you information on destiny mall, weather, stocks and do amazon and google searches. Ask me something : ", "y")
    audio = r.listen(source)
    try:
        text = r.recognize_google(audio)
        print("You said: {}".format(text))
        #several if cases to do different actions when spoken to
        
        
        #Find information about Destiny USA
        if "destiny" in text or "Destiny" in text or "mall" in text:
            say_this("Would you like to know about the stores, outlets, dining options or location of Destiny USA?", "y")
            audio = r.listen(source)
            #try:
            text = r.recognize_google(audio)
            print("You said: {}".format(text))
                
                #Several if cases to perform different Destiny information tasks
            if "store" in text or "stores" in text or "Store" in text or "Stores" in text:
                say_this("Is there a specific store i should look for?",'y')
                audio = r.listen(source)
                try: 
                    text = r.recognize_google(audio)
                    print("You said: {}".format(text))
                
                    user_input = text.title()
                    print(user_input)
                    try:
                        display(data[0].loc[user_input])
                        say_this("Here is the store information for %s, %s"% (user_input,data[0].loc[user_input]), 'n')
                            

                    except IndexError:
                        print("could not find your store")
                        print("Here's a comprehensive list of all the stores in the mall")
                        display(data[0])

                                
                except:
                    print("Could not hear you or could not find your store")
                    print("Here's a comprehensive list of all the stores in the mall")
                    display(data[0])

                    
            
            elif "Outlets" in text or "Outlet" in text or "outlets" in text or "outlet" in text:
                say_this("Is the a specific outlet i should search for?", "y")
                audio = r.listen(source)
                
                try:
                    text = r.recognize_google(audio)
                    print("You said: {}".format(text))
                    user_input = text.title()
                    print(user_input)
                    
                    try:
                        display(data[1].loc[user_input])
                        say_this("Here is the store information for %s, %s"% (user_input,data[0].loc[user_input]), 'n')
                            
                        
                    except IndexError:
                        print("could not find this outlet")
                        
                except:
                    print("Could not hear you")
                    print("could not find this outlet")
                    say_this("Here's a comprehensive list of all outlets in the mall", "y")
                    display(data[1])
                    
                    
            elif "dining options" in text or "food" in text:
                say_this("Is the a specific restaurant i should search for?", "y")
                audio = r.listen(source)
                
                try:
                    text = r.recognize_google(audio)
                    print("You said: {}".format(text))
                    user_input = text.title()
                    print(user_input)
                    
                    try:
                        display(data[2].loc[user_input])
                        say_this("Here is the store information for %s, %s"% (user_input,data[2].loc[user_input]), 'n')
                    except IndexError:
                        print("could not find this outlet")
                        
                except:
                    print("Could not hear you")
                    say_this("Here's a comprehensive list of all restaurants in the mall", "y")
                    display(data[2])
                    
            elif "Location" in text or "location" in text:
                say_this("Destiny address is 9090 Destiny USA Dr, Syracuse, NY 13204","y")
                
            else:
                say_this("I could not find what you are looking for", "y")
                
            
            
        #Find a stock from the Nasdaq using stocksymbol
        elif "stock" in text:
            say_this("say What stock symbol should i look for? : ", "y")
            audio = r.listen(source)
            try:
                text = r.recognize_google(audio)
                print("You said: {}".format(text))
                stock_symbol = text
                find_stock_info(stock_symbol)
            except: 
                print("Could not find stock")
                
                
        #find current local time
        elif "time" in text:
            say_this("Would you like to know what time it is now? :", "y")
            audio = r.listen(source)
            try:
                text = r.recognize_google(audio)
                print("You said: {}".format(text))
                if text != "no":
                    print(time.asctime())
                else:
                    say_this("Sorry i must have misheard", "y")
            except:
                 print("Could not hear a yes or no")
                    
        elif "weather" in text or "Weather" in text:
            get_local_weather()
            
        elif "Google" in text or "google" in text:
            say_this("Would you like me to do a google search?", "y")
            audio = r.listen(source)
            try:
                text = r.recognize_google(audio)
                print("You said: {}".format(text))
                if text != "no":
                    say_this("What should i search for?", "y")
                    audio = r.listen(source) 
                    try:
                        text = r.recognize_google(audio)
                        print("You said: {}".format(text))
                        webbrowser.open("https://www.google.com/search?q= %s" %(text))
                    except:
                        print("could not hear you")
            except:
                print("could not hear you")
                
        elif "Amazon" in text or "amazon" in text:
            say_this("Would you like me to do an amazon search for a product?", "y")
            audio = r.listen(source)
            try:
                text = r.recognize_google(audio)
                print("You said: {}".format(text))
                if text != "no":
                    say_this("What should i search for?", "y")
                    audio = r.listen(source) 
                    try:
                        text = r.recognize_google(audio)
                        print("You said: {}".format(text))
                        webbrowser.open("https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%%3Daps&field-keywords=%s"%(text))
                    except:
                        print("could not hear you")
            except:
                print("could not hear you")
                    
            
    except:
        print("You did not say anything")
        

Hi my Name is Digimall. I can give you information on destiny mall, weather, stocks and do amazon and google searches. Ask me something : 
You said: let me see Destiny Mall
Would you like to know about the stores, outlets, dining options or location of Destiny USA?
You said: show me the dining options
Is the a specific restaurant i should search for?
You said: no
No
Could not hear you
Here's a comprehensive list of all restaurants in the mall


,Hours
Name,
Arby's,Mon.-Thu. 8 a.m.-9:30 p.m. Fri.-Sat. 8 a.m.-10...
Auntie Anne's Pretzels,Mon.-Sat. 10 a.m.-9:30 p.m. Sun. 11 a.m.-6 p.m.
Boston Market,Mon.-Sat. 10 a.m.-9:30 p.m. Sun. 11 a.m.-6 p.m.
Cajun Cafe,Mon.-Sat. 10 a.m.-9:30 p.m. Sun. 11 a.m.-6 p.m.
Cantina Laredo,"Sun. 11 a.m.-10 p.m., brunch 11 a.m.-3 p.m. Mo..."
Caramel Corn Shoppe,Mon.-Sat. 10 a.m.-9:30 p.m. Sun. 11 a.m.-6 p.m.
Charley's Philly Steaks,Mon.-Sat. 10 a.m.-9:30 p.m. Sun. 11 a.m.-6 p.m.
Cheesecake Factory,Mon.-Thu. 11 a.m.-11 p.m. Fri.-Sat. 11 a.m.-12...
Chicken Now,Mon.-Sat. 10 a.m.-9:30 p.m. Sun. 11 a.m.-6 p.m.


KeyboardInterrupt: 

True

TypeError: 'NoneType' object is not callable